### Mysql in python tutorial

This tutorial is mainly for explaining how to interact with Mysql with python. Most times we will interact with relational database, so we should use python to make something like create, insert, select something actions in SQL.

Before anything more, you should use pip to install mysql first:  `pip install pymysql`. Then we could use pymysql to connect with database, you should reference this website first with Github:[Pymysql github](https://github.com/PyMySQL/PyMySQL), there are some examples that we could use. I will show you step by step.

In [1]:
# import module
import pandas as pd
import numpy as np
import pymysql
import warnings

warnings.simplefilter('ignore')

In [3]:
# first thing we should do is to start mysql server first with local computer
# in fact, I have just made mysql server start with each time we start the computer, 
# you don't have to start the Mysql server.

# then we should make the connection object first, that's with host(ip), port, username, password, etc.
con = pymysql.connect(host='localhost', port=3306, user='root', password='lugq1990', db='person')

# one thing to notice: host for local computer is `localhost`, port is default with 3306, then the user is always with 'root'
# with password, db means which database we would to use.

In [5]:
# then we have get the connection, we could do something selection first,
# like what tables do we have?
# do selection with pandas' dataframe is really easy way! as the result is a dataframe 

# we could just use read_sql() function to get the result
# but we should construct the sql we want to use
# pass already connected object with mysql
table_sql = "show tables"
tables_df = pd.read_sql(table_sql, con=con)

# show the result
# then you could see that for my computer, for database 'person', we have two tables
tables_df.head()

,Tables_in_person
0,purchase
1,users


### One thing to notice

Here I just make an example data to insert to mysql with iris dataset, and we could do some actions with SQL syntax. I will show you step by step.

In [6]:
from sklearn.datasets import load_iris

x, y = load_iris(return_X_y=True)

# here I just combine the data and label to be one ndarray
data = np.concatenate((x, y[:, np.newaxis]), axis=1)

print("Get data shape: ", data.shape)

Get data shape:  (150, 5)


### Create table

Here just to make an example to create a table with connection.

In [43]:
create_table_sql = """
create table iris_data (
`f1` varchar(128) not null,
`f2` varchar(128) not null,
`f3` varchar(128) not null,
`f4` varchar(128) not null,
`label` varchar(128)  not null
) 
"""

# every time we use pymysql, we should use try. catch. to get the exception when we do something wrong
# with sql, than we wouldn't get the wrong data to be inserted into table
try:
    with con.cursor() as cursor:
        cursor.execute(create_table_sql)
        
    # with execute, the commit to database doesn't start
    # we have to call commit() function manually
    con.commit()
    print("Table created!")
except Exception as e:
    print("create table with error: %s" % e)

Table created!


In [20]:
# without error, then we could check with this database
tables_df = pd.read_sql(table_sql, con=con)

# as you could see that there are 3 tables, the table we create called 'iris_data' is created.
tables_df.head()

,Tables_in_person
0,iris_data
1,purchase
2,users


In [41]:
# As I will sho you more than just create table, so I don't want to write so many 
# try catch with connection, so I just create a function to execute each SQL.

def execute_sql(sql):
    try:
        with con.cursor() as cursor:
            print("Now is to execute SQL: %s" % sql)
            cursor.execute(sql)
            
        # commit the sql logic to Mysql
        con.commit()
        print("Execute sql : %s finished without error!" % sql)
    except Exception as e:
        print("When execute SQL: %s with error: %s" % (sql, e))

### Insert

In [44]:
# then we could do insert data to the table
# here I make the insert operation
try:
    with con.cursor() as cursor:
        # here we have to insert the data one by one!
        for d in data:
            insert_sql = "insert into `iris_data` (`f1`, `f2`, `f3`, `f4`, `label`) values (%s, %s, %s, %s, %s)"
            cursor.execute(insert_sql, (str(d[0]), str(d[1]), str(d[2]), str(d[3]), str(d[4])))
        
    # then we commit the result to database
    con.commit()
    print("Data inserted!")
except Exception as e:
    print("When insert data to table with error: %s" % (e))


Data inserted!


### Select

In [45]:
select_sql = "select * from iris_data"
df = pd.read_sql(select_sql, con=con)

# get sample data, as you could see that we do insert the data into Mysql table
df.head()

,f1,f2,f3,f4,label
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0


In [46]:
# we could get max value group by label
group_sql = "select max(f1), label from iris_data group by label"

group_df = pd.read_sql(group_sql, con=con)

group_df.head()

,max(f1),label
0,5.8,0.0
1,7.0,1.0
2,7.9,2.0


In [48]:
# get how many records
pd.read_sql("select count(1) as n from iris_data", con=con).head()

,n
0,150


In [49]:
# get distinct label
pd.read_sql("select distinct(label) from iris_data", con=con).head()

,label
0,0.0
1,1.0
2,2.0


In [50]:
# combine two columns
pd.read_sql("select f1 + f2 as f_new from iris_data", con=con).head()

,f_new
0,8.6
1,7.9
2,7.9
3,7.7
4,8.6


### Drop table

In [51]:
drop_sql = 'drop table iris_data'

execute_sql(drop_sql)

# as you could see that we have just dropped the table that we have created!
pd.read_sql(table_sql, con=con).head()

Now is to execute SQL: drop table iris_data
Execute sql : drop table iris_data finished without error!


,Tables_in_person
0,purchase
1,users


There are really many things that we could do with Mysql, here I just put some examples with python and mysql. So you could see that with python, we could really easy to get the data that we could use for later data preprocessing step, right?

The only thing that we should pay attention to is the SQL logic we want to get.